In [1]:
#get a job name and acccess parameters to have a job
import sys
from awsglue.utils import getResolvedOptions

#access sparkcontext in order to get a glueContext
from pyspark.context import SparkContext

#Takes running glue context into a python script
from awsglue.context import GlueContext

glueContext = GlueContext(SparkContext.getOrCreate())
session = glueContext.spark_session
s3_write_path = "s3://joanpydeequ/tmp/demo_anomaly_detection.json"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
95,application_1595892420059_0096,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Extract 
previous_dyf = glueContext.create_dynamic_frame.from_catalog(database="anomaly_detection", table_name="prevdata_csv")
curr_dyf = glueContext.create_dynamic_frame.from_catalog(database="anomaly_detection", table_name="currdata_csv")

#Turn dynamic frame to dataframe to use w/ PyDeequ
previous_dataframe = previous_dyf.toDF()
curr_dataframe = curr_dyf.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import pydeequ
from pydeequ.repository import *
from pydeequ.verification import *

metricsRepository = FileSystemMetricsRepository(session, s3_write_path)

key_tags = {'tag': 'yesterday'}
resultKey_yesterday = ResultKey(session, ResultKey.current_milli_time(), key_tags)

prev_Result = VerificationSuite(session).onData(previous_dataframe)\
    .useRepository(metricsRepository) \
    .saveOrAppendResult(resultKey_yesterday) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateIncrease=2.0), Size())\
    .run()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
metrics_dataframe = metricsRepository.load().forAnalyzers([Size()]).getSuccessMetricsAsDataFrame()

metrics_dataframe.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+----+-----+-------------+---------+
| entity|instance|name|value| dataset_date|      tag|
+-------+--------+----+-----+-------------+---------+
|Dataset|       *|Size|  2.0|1597342948742|yesterday|
+-------+--------+----+-----+-------------+---------+

In [5]:
key_tags = {'tag': 'today'}
resultKey_today = ResultKey(session, ResultKey.current_milli_time(), key_tags)

currResult = VerificationSuite(session).onData(curr_dataframe) \
    .useRepository(metricsRepository) \
    .saveOrAppendResult(resultKey_today) \
    .addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateIncrease=2.0), Size()) \
    .run()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = VerificationResult.checkResultsAsDataFrame(session, currResult)

if (currResult.status != "Success"):
    print("Anomaly for size has been detected")
    print(df.select('constraint').collect())
    metrics_dataframe = metricsRepository.load().forAnalyzers([Size()]).getSuccessMetricsAsDataFrame()
    metrics_dataframe.show()
    
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Anomaly for size has been detected
[Row(constraint='AnomalyConstraint(Size(None))')]
+-------+--------+----+-----+-------------+---------+
| entity|instance|name|value| dataset_date|      tag|
+-------+--------+----+-----+-------------+---------+
|Dataset|       *|Size|  2.0|1597342948742|yesterday|
|Dataset|       *|Size|  5.0|1597342951965|    today|
+-------+--------+----+-----+-------------+---------+